In [27]:
import numpy as np
from sklearn.decomposition import PCA
# 테스트할 데이터
X=np.array([
    [1,1,1,1,1,1,1,1,1,1,3,3,3,3,3,4,5,6],
    [1,2,1,1,1,1,1,1,1,1,3,4,3,3,3,4,5,6],
    [3,3,3,3,3,1,1,1,1,1,1,1,1,1,1,5,4,6],
    [3,4,3,3,3,1,2,1,1,1,1,1,1,1,1,5,4,5],
    [1,1,1,1,1,3,3,3,3,3,1,1,1,1,1,6,4,5],
    [1,2,1,1,1,3,3,3,2,3,1,1,1,1,1,5,4,5],
])
# 주성분 분석(3개의 주성분으로 축소)
pca=PCA(n_components=3)
X2D=pca.fit_transform(X)
X2D[:5]

array([[ 3.84432202,  0.20850925,  0.46823094],
       [ 4.09935419, -0.13332384, -0.49268129],
       [-1.70050345, -3.04924012,  0.76869479],
       [-2.21379239, -3.06986255, -0.66041977],
       [-2.102115  ,  3.36337224,  0.55345806]])

In [28]:
# 주성분 1은 54%, 주성분 2는 42%, 주성분 3은 2%, 나머지가 2% (주성분 1,2만으로 96%를 설명함)
# 각 주성분의 축에 해당하는 데이터셋의 분산 비율
for i in pca.explained_variance_ratio_:
    print(f'{i:.2f}')

0.54
0.42
0.02


In [2]:
# 3개의 주성분으로 축소된 자료를 원래의 형태로 복원(정보의 손실이 있음)
X3D_inv=pca.inverse_transform(X2D)
X3D_inv[0]
# 복원 단계에서 정보 손실이 발생할 수 있음

array([1.01080967, 1.1237144 , 1.01080967, 1.01080967, 1.01080967,
       1.04652366, 0.87642192, 1.04652366, 1.15635772, 1.04652366,
       2.94266667, 3.3355072 , 2.94266667, 2.94266667, 2.94266667,
       4.16176255, 4.97133333, 6.14683992])

In [3]:
# 재구성 오차(reconstruction error) 계산 : 원본 데이터와 압축 후 원복한 데이터 사이의 평균 제곱 거리
1 - pca.explained_variance_ratio_.sum()

0.015189685531841413

In [4]:
# MNIST 압축
from sklearn.datasets import fetch_openml
mnist=fetch_openml('mnist_784')

In [5]:
from sklearn.model_selection import train_test_split

X=mnist['data']
y=mnist['target']

X_train, X_test, y_train, y_test=train_test_split(X, y)

In [6]:
# 적절한 차원의 수 선택
# 분산을 95%로 유지하는 차원의 수 계산
from sklearn.decomposition import PCA

pca=PCA()
pca.fit(X_train)
cumsum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(cumsum >= 0.95) + 1
d

154

In [7]:
# 분산 비율을 직접 지정하는 방식
pca=PCA(n_components=0.95)
X_reduced=pca.fit_transform(X_train)
print(pca.n_components_)
print(np.sum(pca.explained_variance_ratio_))

154
0.9503238973449987


In [8]:
# 154 차원으로 압축
pca=PCA(n_components=154)
X_reduced=pca.fit_transform(X_train)
# 784 차원으로 복원
X_recovered=pca.inverse_transform(X_reduced)

In [9]:
# 대량의 데이터의 경우 pca를 구현하기 위해 전체 데이터셋을 메모리에 올리는 것이 어려울 수 있음
# 점진적 pca(Incremental PCA) 알고리즘을 사용하여 미니배치 방법으로 pca를 실행할 수 있음
from sklearn.decomposition import IncrementalPCA

n_batches=100
inc_pca=IncrementalPCA(n_components=154)
# 미니배치에 해당하는 부분만 사용하므로 메모리가 절약됨
for X_batch in np.array_split(X_train, n_batches):
    print(".", end="")
    inc_pca.partial_fit(X_batch)

X_reduced=inc_pca.transform(X_train)
X_recovered_inc_pca=inc_pca.inverse_transform(X_reduced)

....................................................................................................

In [10]:
# 일반 PCA와 점진적 PCA로 MNIST 데이터를 변환한 결과 비교
# 평균이 같은지 확인
# allclose() : 두 배열이 오차범위 내에서 같으면 True, 다르면 False
np.allclose(pca.mean_, inc_pca.mean_)

True

In [11]:
# 학습시간 비교(랜덤 포레스트와 로지스특 회귀분석, 원본 데이터와 압축된 데이터 비교)
X_train=mnist['data'][:60000]
y_train=mnist['target'][:60000]

X_test=mnist['data'][60000:]
y_test=mnist['target'][60000:]

In [12]:
# 랜덤 포레스트 모형
from sklearn.ensemble import RandomForestClassifier

rnd_clf=RandomForestClassifier(n_estimators=10, random_state=42)

In [13]:
%%time
rnd_clf.fit(X_train, y_train)

Wall time: 4.24 s


RandomForestClassifier(n_estimators=10, random_state=42)

In [14]:
# 정확도 출력
from sklearn.metrics import accuracy_score

y_pred=rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9492

In [15]:
# PCA를 사용하여 분산이 95%가 되도록 차원 축소
from sklearn.decomposition import PCA

pca=PCA(n_components=0.95)
X_train_reduced=pca.fit_transform(X_train)

In [17]:
import time
# 랜덤 포레스트 모형에 압축된 데이터 입력
rnd_clf2=RandomForestClassifier(n_estimators=10, random_state=42)
t0=time.time()
rnd_clf2.fit(X_train_reduced, y_train)
t1=time.time()
print(f"학습용 데이터셋(압축)의 학습 시간 : {t1-t0:.2f}s")
# 차원 축소가 반드시 학습 시간 단축을 의미하지는 않음

학습용 데이터셋(압축)의 학습 시간 : 9.12s


In [18]:
# 검증용 데이터셋으로 평가
X_test_reduced=pca.transform(X_test)
y_pred=rnd_clf2.predict(X_test_reduced)
accuracy_score(y_test, y_pred)
# 차원 축소로 인한 정보 손실로 성능이 감소되는 것이 일반적

0.9009

In [22]:
from sklearn.linear_model import LogisticRegression

log_clf=LogisticRegression(multi_class='multinomial', max_iter=300, random_state=42)
# multinomial : 소프트맥스 방식의 로지스틱 회귀분석, 시간이 많이 걸림
t0=time.time()
log_clf.fit(X_train, y_train)
t1=time.time()
print(f"학습용 데이터셋(원본)의 학습 시간 : {t1-t0:.2f}s")

학습용 데이터셋(원본)의 학습 시간 : 38.62s


D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
y_pred=log_clf.predict(X_test)
accuracy_score(y_test, y_pred)
# 랜덤포레스트보다는 성능이 좋음

0.9226

In [24]:
# 주성분 분석으로 축소된 데이터셋으로 학습
log_clf2=LogisticRegression(multi_class="multinomial", max_iter=1000, random_state=42)
t0=time.time()
log_clf2.fit(X_train_reduced, y_train)
t1=time.time()
print(f"학습용 데이터셋(압축)의 학습 시간 : {t1-t0:.2f}s")

학습용 데이터셋(압축)의 학습 시간 : 39.62s


D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
y_pred=log_clf2.predict(X_test_reduced)
accuracy_score(y_test, y_pred)

0.9239